Загружаем библиотеки

In [ ]:
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV, cross_val_score

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

matplotlib.rcParams.update({'font.size': 14})

Загружаем данные

In [168]:
train_ds = pd.read_csv('C:/Users/Glsteel/train.csv')
test_ds = pd.read_csv('C:/Users/Glsteel/test.csv')

Изучаем датасет.

In [97]:
train_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.5+ MB


In [98]:
test_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       3959 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null object
Ecology_3        5000 non-null object
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Healthcare_1     2623 non-null float64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null object
dtypes: float64(7), int64(9), object(3)
memory usage: 742.3+ KB


In [100]:
train_object_features = train_ds.select_dtypes(include=['object']) #выделяем количественные переменные
train_num_features = train_ds.select_dtypes(exclude=['object']) #выделяем категориальные переменные
train_object_features.head()

,Id,Ecology_2,Ecology_3,Shops_2
0,14038,B,B,B
1,15053,B,B,B
2,4765,B,B,B
3,5809,B,B,B
4,10783,B,B,B


In [101]:
train_object_features['Shops_2'].value_counts()
train_object_features['Ecology_2'].value_counts()
train_object_features['Ecology_3'].value_counts()

B    9725
A     275
Name: Ecology_3, dtype: int64

In [102]:
train_num_features.describe()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [103]:
test_ds.describe()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,3959.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,2623.000000,5000.00000,5000.000000
mean,51.279200,1.910000,56.449500,36.158810,5.976800,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1146.657263,1.31940,4.242800
std,44.179466,0.838594,19.092787,17.825287,9.950018,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1044.744231,1.47994,4.777365
min,0.000000,0.000000,1.378543,0.333490,0.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.00000,0.000000
25%,21.000000,1.000000,41.906231,23.092026,1.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,325.000000,0.00000,1.000000
50%,37.000000,2.000000,52.921340,32.925087,6.000000,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,900.000000,1.00000,3.000000
75%,77.000000,2.000000,66.285129,45.174091,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,1548.000000,2.00000,6.000000
max,212.000000,17.000000,223.453689,303.071094,620.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.00000,23.000000


In [178]:
len(train_ds) - train_ds.count() #определяем нулевые значения 

Id               0
DistrictId       0
Rooms            0
Square           0
LifeSquare       0
KitchenSquare    0
Floor            0
HouseFloor       0
HouseYear        0
Ecology_1        0
Ecology_2        0
Ecology_3        0
Social_1         0
Social_2         0
Social_3         0
Healthcare_1     0
Helthcare_2      0
Shops_1          0
Shops_2          0
Price            0
dtype: int64

Заменяем нулевые значения, корректируем выбросы

In [171]:
train_ds.loc[train_ds['LifeSquare'].isnull(), 'LifeSquare'] = train_ds['Square'] - train_ds['KitchenSquare']
test_ds.loc[test_ds['LifeSquare'].isnull(), 'LifeSquare'] = test_ds['Square'] - test_ds['KitchenSquare']
#убрали нулевые значения признака "lifeSquare" путем вычитания Площади кухни из Общей площади

In [172]:
train_ds.loc[train_ds['Healthcare_1'].isnull(), 'Healthcare_1'] = train_ds['Healthcare_1'].median
test_ds.loc[test_ds['Healthcare_1'].isnull(), 'Healthcare_1'] = test_ds['Healthcare_1'].median
#приравняли пропущенные значения признака "Healthcare_1" к медианному значению

In [173]:
train_ds.loc[train_ds['Rooms'] > 5, 'Rooms'] = train_ds['Rooms'].median()
train_ds.loc[train_ds['Rooms'] == 0, 'Rooms'] = train_ds['Rooms'].median()
test_ds.loc[test_ds['Rooms'] > 5, 'Rooms'] = test_ds['Rooms'].median()
test_ds.loc[test_ds['Rooms'] == 0, 'Rooms'] = test_ds['Rooms'].median()
#приравняли количество комнат равное нулю и большее 5 к моде.

In [174]:
train_ds.loc[train_ds['Square'] > 200, 'Square'] = train_ds['Square'].median()
train_ds.loc[train_ds['Square'] < 20, 'Square'] = train_ds['Square'].median()
test_ds.loc[test_ds['Square'] > 200, 'Square'] = test_ds['Square'].median()
test_ds.loc[test_ds['Square'] < 20, 'Square'] = test_ds['Square'].median()
#убираем выбросы по признаку Площадь

In [175]:
train_ds.loc[train_ds['LifeSquare'] > 200, 'LifeSquare'] = train_ds['LifeSquare'].median()
train_ds.loc[train_ds['LifeSquare'] < 10, 'LifeSquare'] = train_ds['LifeSquare'].median()
test_ds.loc[test_ds['LifeSquare'] > 200, 'LifeSquare'] = test_ds['LifeSquare'].median()
test_ds.loc[test_ds['LifeSquare'] < 10, 'LifeSquare'] = test_ds['LifeSquare'].median()
#убираем выбросы по признаку Жилая площадь

In [176]:
train_ds.loc[train_ds['KitchenSquare'] > 25, 'KitchenSquare'] = train_ds['KitchenSquare'].median()
test_ds.loc[test_ds['KitchenSquare'] > 25, 'KitchenSquare'] = test_ds['KitchenSquare'].median()
#убираем выбросы по признаку Площадь кухни

In [177]:
train_ds.loc[train_ds['LifeSquare'] > train_ds['Square'], 'LifeSquare'] = train_ds['Square'] - train_ds['KitchenSquare']
test_ds.loc[test_ds['LifeSquare'] > test_ds['Square'], 'LifeSquare'] = test_ds['Square'] - test_ds['KitchenSquare']
#убираем случаи, когда жилая площадь больше общей                                                                                   

In [113]:
train_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null object
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       10000 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     10000 non-null object
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(7), int64(8), object(5)
memory usage: 1.5+ MB


In [114]:
test_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
Id               5000 non-null object
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       5000 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null object
Ecology_3        5000 non-null object
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Healthcare_1     5000 non-null object
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null object
dtypes: float64(6), int64(8), object(5)
memory usage: 742.3+ KB


In [62]:
test_ds.describe()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000
mean,51.279200,1.906200,56.511740,41.315858,5.700800,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1.31940,4.242800
std,44.179466,0.805935,18.623536,18.892673,3.821282,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1.47994,4.777365
min,0.000000,1.000000,20.011215,10.692499,0.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.00000,0.000000
25%,21.000000,1.000000,41.977760,27.830435,1.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,0.00000,1.000000
50%,37.000000,2.000000,52.920459,36.931498,6.000000,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,1.00000,3.000000
75%,77.000000,2.000000,66.269760,49.749120,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,2.00000,6.000000
max,212.000000,5.000000,189.679576,168.729035,24.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,6.00000,23.000000


In [179]:
train_ds.drop(['Id'],axis=1,inplace=True)
train_ds.drop(['Ecology_2'],axis=1,inplace=True)
train_ds.drop(['Ecology_3'],axis=1,inplace=True)
train_ds.drop(['Shops_2'],axis=1,inplace=True)
train_ds.drop(['Healthcare_1'],axis=1,inplace=True)

test_ds.drop(['Ecology_2'],axis=1,inplace=True)
test_ds.drop(['Ecology_3'],axis=1,inplace=True)
test_ds.drop(['Shops_2'],axis=1,inplace=True)
test_ds.drop(['Healthcare_1'],axis=1,inplace=True)

In [152]:
train_ds.keys()

Index(['DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor',
       'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2',
       'Social_3', 'Helthcare_2', 'Shops_1', 'Price'],
      dtype='object')

In [153]:
feature_names = ['DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor',
                 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2',
                 'Social_3', 'Helthcare_2', 'Shops_1']

target_name = 'Price'

In [154]:
train_ds.head()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Price
0,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,33,7976,5,0,11,184966.930730
1,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,46,10309,1,1,16,300009.450063
2,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,34,7759,0,1,3,220925.908524
3,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,23,5735,3,0,5,175616.227217
4,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,35,5776,1,2,4,150226.531644


In [155]:
X = pd.DataFrame(train_ds, columns=['DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor',
       'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2',
       'Social_3', 'Helthcare_2', 'Shops_1'])
X.head()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1
0,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,33,7976,5,0,11
1,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,46,10309,1,1,16
2,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,34,7759,0,1,3
3,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,23,5735,3,0,5
4,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,35,5776,1,2,4


In [156]:
y = pd.DataFrame(train_ds, columns=[target_name])
y.head(5)

,Price
0,184966.930730
1,300009.450063
2,220925.908524
3,175616.227217
4,150226.531644


Работаем с тренировочной выборкой

In [157]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [158]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [77]:
#Осуществляем выбор гиперпараметров.
rf_mod_1 = RandomForestRegressor(random_state=42)

random_grid = {
    'n_estimators': np.arange(200, 501, 20),
    'max_depth': np.arange(2, 51, 2),
    'max_features': [0.5, 0.6, 0.7, 0.8, 0.9],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10]
}

random_search = RandomizedSearchCV(
    estimator=rf_mod_1,
    param_distributions=random_grid,
    n_iter=50,
    scoring='r2',
    cv=10,
    verbose=2,
    random_state=42
)

random_search.fit(X_train_scaled, y_train)

print(random_search.best_score_)
print(random_search.best_params_)

Fitting 10 folds for each of 50 candidates, totalling 500 fits
[CV] n_estimators=260, min_samples_split=2, min_samples_leaf=4, max_features=0.9, max_depth=44 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=260, min_samples_split=2, min_samples_leaf=4, max_features=0.9, max_depth=44, total=   5.2s
[CV] n_estimators=260, min_samples_split=2, min_samples_leaf=4, max_features=0.9, max_depth=44 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.4s remaining:    0.0s


[CV]  n_estimators=260, min_samples_split=2, min_samples_leaf=4, max_features=0.9, max_depth=44, total=   5.2s
[CV] n_estimators=260, min_samples_split=2, min_samples_leaf=4, max_features=0.9, max_depth=44 
[CV]  n_estimators=260, min_samples_split=2, min_samples_leaf=4, max_features=0.9, max_depth=44, total=   5.3s
[CV] n_estimators=260, min_samples_split=2, min_samples_leaf=4, max_features=0.9, max_depth=44 
[CV]  n_estimators=260, min_samples_split=2, min_samples_leaf=4, max_features=0.9, max_depth=44, total=   5.2s
[CV] n_estimators=260, min_samples_split=2, min_samples_leaf=4, max_features=0.9, max_depth=44 
[CV]  n_estimators=260, min_samples_split=2, min_samples_leaf=4, max_features=0.9, max_depth=44, total=   5.2s
[CV] n_estimators=260, min_samples_split=2, min_samples_leaf=4, max_features=0.9, max_depth=44 
[CV]  n_estimators=260, min_samples_split=2, min_samples_leaf=4, max_features=0.9, max_depth=44, total=   5.3s
[CV] n_estimators=260, min_samples_split=2, min_samples_leaf=

...

[CV]  n_estimators=420, min_samples_split=10, min_samples_leaf=2, max_features=0.9, max_depth=44, total=   9.7s
[CV] n_estimators=420, min_samples_split=10, min_samples_leaf=2, max_features=0.9, max_depth=44 
[CV]  n_estimators=420, min_samples_split=10, min_samples_leaf=2, max_features=0.9, max_depth=44, total=   9.7s
[CV] n_estimators=420, min_samples_split=10, min_samples_leaf=2, max_features=0.9, max_depth=44 
[CV]  n_estimators=420, min_samples_split=10, min_samples_leaf=2, max_features=0.9, max_depth=44, total=   9.8s
[CV] n_estimators=420, min_samples_split=10, min_samples_leaf=2, max_features=0.9, max_depth=44 
[CV]  n_estimators=420, min_samples_split=10, min_samples_leaf=2, max_features=0.9, max_depth=44, total=  10.0s
[CV] n_estimators=420, min_samples_split=10, min_samples_leaf=2, max_features=0.9, max_depth=44 
[CV]  n_estimators=420, min_samples_split=10, min_samples_leaf=2, max_features=0.9, max_depth=44, total=   9.7s
[CV] n_estimators=420, min_samples_split=10, min_sam

[CV]  n_estimators=460, min_samples_split=10, min_samples_leaf=4, max_features=0.5, max_depth=42, total=   6.8s
[CV] n_estimators=460, min_samples_split=10, min_samples_leaf=4, max_features=0.5, max_depth=42 
[CV]  n_estimators=460, min_samples_split=10, min_samples_leaf=4, max_features=0.5, max_depth=42, total=   7.9s
[CV] n_estimators=460, min_samples_split=10, min_samples_leaf=4, max_features=0.5, max_depth=42 
[CV]  n_estimators=460, min_samples_split=10, min_samples_leaf=4, max_features=0.5, max_depth=42, total=   6.6s
[CV] n_estimators=460, min_samples_split=10, min_samples_leaf=4, max_features=0.5, max_depth=42 
[CV]  n_estimators=460, min_samples_split=10, min_samples_leaf=4, max_features=0.5, max_depth=42, total=   6.4s
[CV] n_estimators=460, min_samples_split=10, min_samples_leaf=4, max_features=0.5, max_depth=42 
[CV]  n_estimators=460, min_samples_split=10, min_samples_leaf=4, max_features=0.5, max_depth=42, total=   6.5s
[CV] n_estimators=460, min_samples_split=10, min_sam

[CV]  n_estimators=460, min_samples_split=2, min_samples_leaf=1, max_features=0.7, max_depth=4, total=   2.8s
[CV] n_estimators=460, min_samples_split=2, min_samples_leaf=1, max_features=0.7, max_depth=4 
[CV]  n_estimators=460, min_samples_split=2, min_samples_leaf=1, max_features=0.7, max_depth=4, total=   2.7s
[CV] n_estimators=460, min_samples_split=2, min_samples_leaf=1, max_features=0.7, max_depth=4 
[CV]  n_estimators=460, min_samples_split=2, min_samples_leaf=1, max_features=0.7, max_depth=4, total=   2.9s
[CV] n_estimators=460, min_samples_split=2, min_samples_leaf=1, max_features=0.7, max_depth=4 
[CV]  n_estimators=460, min_samples_split=2, min_samples_leaf=1, max_features=0.7, max_depth=4, total=   2.8s
[CV] n_estimators=460, min_samples_split=2, min_samples_leaf=1, max_features=0.7, max_depth=4 
[CV]  n_estimators=460, min_samples_split=2, min_samples_leaf=1, max_features=0.7, max_depth=4, total=   2.8s
[CV] n_estimators=460, min_samples_split=2, min_samples_leaf=1, max_fe

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed: 51.3min finished


0.735945501348817
{'n_estimators': 400, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 0.5, 'max_depth': 26}


In [159]:
# обучаем подобранную модель на всех тренировочных данных и проверяем качество.

scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)

rfr_mod = RandomForestRegressor(
    n_estimators=400,
    max_depth=26,
    max_features=0.5,
    min_samples_leaf=1,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)

rfr_mod.fit(X_scaled, y)

scores = cross_val_score(model, X_scaled, y, scoring='r2', cv=10, n_jobs=-1)

print(f'cv mean: {np.round(np.mean(scores), 4)}')
print(f'cv std:  {np.round(np.std(scores), 4)}')

#y_pred_rfr = rfr_mod.predict(X)
#r2(y, y_pred_rfr)

cv mean: 0.7413
cv std:  0.024


Работаем с тестовой выборкой

In [160]:
X_test = pd.DataFrame(test_ds, columns=['DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor',
       'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2',
       'Social_3', 'Helthcare_2', 'Shops_1'])
X_test.head()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1
0,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,11,2748,1,0,0
1,74,2.0,69.263183,68.263183,1.0,6,1.0,1977,0.075779,6,1437,3,0,2
2,190,1.0,52.919578,15.948246,12.0,2,5.0,1909,0.000000,30,7538,87,5,5
3,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,23,4583,3,3,3
4,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,2,629,1,0,0


In [161]:
y_test = pd.DataFrame(train_ds, columns=[target_name])
y_test.head()

,Price
0,184966.930730
1,300009.450063
2,220925.908524
3,175616.227217
4,150226.531644


In [162]:
# масштабируем новые данные подготовленным на тренировочных данных объектом класса MinMaxScaler

X_test_scaled = scaler.transform(X_test)

In [163]:
# предсказываем цены квартир для новых наблюдений
y_test_pred = rfr_mod.predict(X_test_scaled)

In [180]:
test_ds.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.906200,56.511740,41.315858,5.700800,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1.31940,4.242800
std,4832.674037,44.179466,0.805935,18.623536,18.892673,3.821282,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1.47994,4.777365
min,1.000000,0.000000,1.000000,20.011215,10.692499,0.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.977760,27.830435,1.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.920459,36.931498,6.000000,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.269760,49.749120,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,2.00000,6.000000
max,16795.000000,212.000000,5.000000,189.679576,168.729035,24.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,6.00000,23.000000


In [181]:
# записываем id объекта и предсказанную цену в датасет

predictions = pd.DataFrame({
    'Id': test_ds['Id'],
    'Price': y_test_pred
})
predictions.head()

,Id,Price
0,725,168521.974693
1,15856,227499.529766
2,5480,311188.574695
3,15664,342421.654167
4,14275,142768.552601


In [182]:
#сохраняем результат предсказаний
predictions.to_csv('GAzanov_predictions.csv', sep=',', index=False, encoding='utf-8')
